In [1]:
from pyfirmata import Arduino, util
import time


class Controller:

    def __init__(self):
        self.board = Arduino('COM3')                        # Select Arduino Port
        self.signalPin_x = self.board.get_pin('d:3:p')      # DigitalPin 3
        self.signalPin_y = self.board.get_pin('d:5:p')      # DigitalPin 5
        self.signalPin_z = self.board.get_pin('d:6:p')      # DigitalPin 6

    def analog_out(self, val_x, val_y, val_z):

        self.signalPin_x.write(val_x)
        time.sleep(0.05)

        self.signalPin_y.write(val_y)
        time.sleep(0.05)

        self.signalPin_z.write(val_z)
        time.sleep(0.05)

    def close(self):
        self.board.exit()




In [2]:
%pip install pyfirmata

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pybullet

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import mediapipe as mp
mpDraw = mp.solutions.drawing_utils
mpHands = mp.solutions.hands
hands = mpHands.Hands()


fwidth = 640
fheight = 480


def mean_hand_pos(hand_landmarks): # Function to find mean (x,y,z) position of the hand
    global fwidth, fheight

    idxs = [0, 1, 2, 5, 9, 13, 17]
    x_sum = 0
    y_sum = 0
    z_sum = 0
    for idx in idxs:
        x_sum += hand_landmarks[idx].x
        y_sum += hand_landmarks[idx].y
        z_sum += hand_landmarks[idx].z

    x_mean_norm = x_sum/len(idxs)
    y_mean_norm = y_sum/len(idxs)
    z_mean_norm = z_sum/len(idxs)

    x_mean = x_mean_norm * fwidth
    y_mean = y_mean_norm * fheight

    return int(x_mean), int(y_mean), z_mean_norm


def interp(x, in_min, in_max, out_min, out_max): # Arduino Map Function
    return ((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)


def gesture_control(cap):
    global fwidth, fheight

    x_out_mc = 0
    y_out_mc = 0
    z_out_mc = 0
    x_out_sim = 0
    y_out_sim = 0
    z_out_sim = 0

    _, img = cap.read()
    img = cv2.flip(img, 1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            x_mean, y_mean, z_norm = mean_hand_pos(list(handLms.landmark))          # Find mean hand pos
            z_mean = int(z_norm*150)
            x_rel = x_mean - int(fwidth/2)                                # Shift origin to center of frame
            y_rel = -1*(y_mean - int(fheight/2))                          # Shift origin to center of frame
            x_out_mc = interp(x_rel, -320, 320, 0, 1)                     # Map position to (0,1) for microcontroller
            y_out_mc = interp(y_rel, -240, 240, 0, 1)                     # Map position to (0,1) for microcontroller
            z_out_mc = -1*(z_norm)
            x_out_sim = interp(x_rel, -320, 320, -1, 1)                   # Map position to (-1,1) for drone_sim
            y_out_sim = interp(y_rel, -320, 320, -1, 1)                   # Map position to (-1,1) for drone_sim
            z_out_sim = z_out_mc
            cv2.circle(img, (x_mean, y_mean), 10, (255, 0, 0), cv2.FILLED)  # Show mean position
            cv2.putText(img, f"X : {x_rel} Y : {y_rel} Z : {z_mean}", (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (209, 80, 0, 255), 3)
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)   # Draw Landmarks

    cv2.line(img, (int(fwidth/2), 0), (int(fwidth/2), fheight), (0, 255, 0), 2) # Draw Gridlines
    cv2.line(img, (0, int(fheight/2)), (fwidth, int(fheight/2)), (0, 255, 0), 2)

    return img, x_out_mc, y_out_mc, z_out_mc, x_out_sim, y_out_sim, z_out_sim


In [6]:
import cv2
from drone_sim import Drone

cap = cv2.VideoCapture(0)
drone = Drone()             #Turn the drone_sim on
#controller = Controller()  #Turn the controller on

while True:

    frame, x_out_mc, y_out_mc, z_out_mc, x_out_sim, y_out_sim, z_out_sim = gesture_control(cap)  # Calculate Voltages and Actions corresponding to Gestures

    drone.show(x_out_sim, y_out_sim, z_out_sim)                 # Show drone sim

    #controller.analog_out(x_out_mc, y_out_mc, z_out_mc)        # Write voltages to microcontroller

    cv2.imshow("Gesture Control", frame)                        # Show the gesture frame

    k = cv2.waitKey(10)
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()
#controller.close()


pybullet build time: Apr 26 2022 03:11:02
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (CML GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.0.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.0.3
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (CML GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
